In [1]:
# %%time

# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [2]:
%%time

! pip install huggify-data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 116.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 134.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

CPU times: user 543 ms, sys: 93.5 ms, total: 637 ms
Wall time: 1min 33s


In [3]:
# import gc
# import os
# import torch
# from datasets import load_dataset
# from peft import LoraConfig, PeftModel, PeftConfig
# from transformers import (
#     AutoModelForCausalLM,
#     AutoTokenizer,
#     BitsAndBytesConfig,
#     TrainingArguments,
#     logging,
# )
# from trl import SFTTrainer


# class LlamaTrainer:
#     """
#     A class to handle the training and uploading of a fine-tuned Llama model using the Hugging Face transformers library.
#     """

#     def __init__(self, model_name: str, dataset_name: str, new_model: str, huggingface_token: str):
#         """
#         Initialize the LlamaTrainer class.

#         Args:
#             model_name (str): The name of the pre-trained model.
#             dataset_name (str): The name of the dataset to use for training.
#             new_model (str): The name of the new fine-tuned model to be saved.
#             huggingface_token (str): The Hugging Face token for authentication.
#         """
#         self.model_name = model_name
#         self.dataset_name = dataset_name
#         self.new_model = new_model
#         self.hf_token = huggingface_token
#         self.device_map = {"": 0}
#         self.bnb_config = self.configure_bitsandbytes()
#         self.tokenizer = self.load_tokenizer()
#         self.model = self.load_model()

#     def configure_bitsandbytes(self) -> BitsAndBytesConfig:
#         """
#         Configure the BitsAndBytes quantization settings.

#         Returns:
#             BitsAndBytesConfig: The configuration object for BitsAndBytes.
#         """
#         compute_dtype = torch.float16
#         bnb_config = BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_quant_type="nf4",
#             bnb_4bit_compute_dtype=compute_dtype,
#             bnb_4bit_use_double_quant=False,
#         )
#         return bnb_config

#     def load_model(self) -> AutoModelForCausalLM:
#         """
#         Load the pre-trained model with quantization settings.

#         Returns:
#             AutoModelForCausalLM: The loaded model.
#         """
#         print("Starting model loading...")
#         model = AutoModelForCausalLM.from_pretrained(
#             self.model_name,
#             quantization_config=self.bnb_config,
#             device_map=self.device_map,
#         )
#         model.config.use_cache = False
#         model.config.pretraining_tp = 1
#         print("Model loading completed.")
#         return model

#     def load_tokenizer(self) -> AutoTokenizer:
#         """
#         Load the tokenizer for the pre-trained model.

#         Returns:
#             AutoTokenizer: The loaded tokenizer.
#         """
#         tokenizer = AutoTokenizer.from_pretrained(
#             self.model_name, trust_remote_code=True
#         )
#         tokenizer.pad_token = tokenizer.eos_token
#         tokenizer.padding_side = "right"
#         return tokenizer

#     def configure_lora(self, lora_r: int = 64, lora_alpha: int = 16, lora_dropout: float = 0.1) -> LoraConfig:
#         """
#         Configure the LoRA (Low-Rank Adaptation) settings.

#         Args:
#             lora_r (int, optional): The rank of the LoRA matrices. Default is 64.
#             lora_alpha (int, optional): The scaling factor for the LoRA matrices. Default is 16.
#             lora_dropout (float, optional): The dropout rate for LoRA layers. Default is 0.1.

#         Returns:
#             LoraConfig: The configuration object for LoRA.
#         """
#         peft_config = LoraConfig(
#             lora_alpha=lora_alpha,
#             lora_dropout=lora_dropout,
#             r=lora_r,
#             bias="none",
#             task_type="CAUSAL_LM",
#         )
#         return peft_config

#     def configure_training_arguments(
#         self,
#         output_dir: str = "./results",
#         num_train_epochs: int = 4,
#         per_device_train_batch_size: int = 2,
#         gradient_accumulation_steps: int = 2,
#         learning_rate: float = 2e-4,
#         weight_decay: float = 0.001,
#         save_steps: int = 0,
#         logging_steps: int = 25,
#         fp16: bool = False,
#         bf16: bool = False,
#         max_grad_norm: float = 0.3,
#         max_steps: int = -1,
#         warmup_ratio: float = 0.03,
#         group_by_length: bool = True,
#         lr_scheduler_type: str = "cosine",
#     ) -> TrainingArguments:
#         """
#         Configure the training arguments.

#         Args:
#             output_dir (str, optional): The output directory for saving results. Default is "./results".
#             num_train_epochs (int, optional): The number of training epochs. Default is 4.
#             per_device_train_batch_size (int, optional): The batch size per device. Default is 2.
#             gradient_accumulation_steps (int, optional): The number of gradient accumulation steps. Default is 2.
#             learning_rate (float, optional): The learning rate for training. Default is 2e-4.
#             weight_decay (float, optional): The weight decay for optimization. Default is 0.001.
#             save_steps (int, optional): The number of steps between each save. Default is 0.
#             logging_steps (int, optional): The number of steps between each log. Default is 25.
#             fp16 (bool, optional): Use FP16 precision. Default is False.
#             bf16 (bool, optional): Use BF16 precision. Default is False.
#             max_grad_norm (float, optional): The maximum gradient norm. Default is 0.3.
#             max_steps (int, optional): The maximum number of training steps. Default is -1 (no limit).
#             warmup_ratio (float, optional): The warmup ratio for the learning rate scheduler. Default is 0.03.
#             group_by_length (bool, optional): Whether to group sequences by length. Default is True.
#             lr_scheduler_type (str, optional): The type of learning rate scheduler. Default is "cosine".

#         Returns:
#             TrainingArguments: The configuration object for training arguments.
#         """
#         training_arguments = TrainingArguments(
#             output_dir=output_dir,
#             num_train_epochs=num_train_epochs,
#             per_device_train_batch_size=per_device_train_batch_size,
#             gradient_accumulation_steps=gradient_accumulation_steps,
#             optim="paged_adamw_32bit",
#             save_steps=save_steps,
#             logging_steps=logging_steps,
#             learning_rate=learning_rate,
#             weight_decay=weight_decay,
#             fp16=fp16,
#             bf16=bf16,
#             max_grad_norm=max_grad_norm,
#             max_steps=max_steps,
#             warmup_ratio=warmup_ratio,
#             group_by_length=group_by_length,
#             lr_scheduler_type=lr_scheduler_type,
#             gradient_checkpointing=True,  # Enable gradient checkpointing
#             report_to="tensorboard",
#         )
#         return training_arguments

#     def train_model(
#         self, training_arguments: TrainingArguments, peft_config: LoraConfig, max_seq_length: int = None, packing: bool = False
#     ):
#         """
#         Train the model with the given dataset and configuration.

#         Args:
#             training_arguments (TrainingArguments): The training arguments.
#             peft_config (LoraConfig): The LoRA configuration.
#             max_seq_length (int, optional): The maximum sequence length for training. Default is None.
#             packing (bool, optional): Whether to use packing for training. Default is False.
#         """
#         print("Starting dataset loading...")
#         dataset = load_dataset(self.dataset_name, split="train")
#         print("Dataset loading completed.")

#         print("Trainer defined...")
#         trainer = SFTTrainer(
#             model=self.model,
#             train_dataset=dataset,
#             peft_config=peft_config,
#             dataset_text_field="text",
#             max_seq_length=max_seq_length,
#             tokenizer=self.tokenizer,
#             args=training_arguments,
#             packing=packing,
#         )

#         try:
#             print("Start training...")
#             trainer.train()
#         except RuntimeError as e:
#             if "out of memory" in str(e):
#                 print(
#                     "CUDA out of memory error occurred, attempting to clear cache and retry."
#                 )
#                 self.clear_cuda_cache()
#                 print("Start training...")
#                 trainer.train()

#         print("Save model...")
#         trainer.model.save_pretrained(self.new_model)
#         self.tokenizer.save_pretrained(self.new_model)
#         print("Training and saving completed.")

#         # Push to hub
#         self.push_model_to_hub(trainer)

#     def push_model_to_hub(self, trainer: SFTTrainer):
#         """
#         Push the trained model to the Hugging Face Hub.

#         Args:
#             trainer (SFTTrainer): The trainer object containing the trained model.
#         """
#         print("Starting model merging and saving...")
#         try:
#             print("Pushing model to the Hugging Face Hub...")
#             self.tokenizer.push_to_hub(self.new_model, use_temp_dir=False)
#             trainer.model.push_to_hub(self.new_model, use_temp_dir=False, token=self.hf_token)
#             print("Model pushing completed.")
#         except RuntimeError as e:
#             if "out of memory" in str(e):
#                 print("CUDA out of memory error occurred, attempting to clear cache and retry.")
#                 self.clear_cuda_cache()
#                 print("Pushing model to the Hugging Face Hub...")
#                 self.tokenizer.push_to_hub(self.new_model, use_temp_dir=False)
#                 trainer.model.push_to_hub(self.new_model, use_temp_dir=False, token=self.hf_token)
#                 print("Model pushing completed.")

#     def clear_cuda_cache(self):
#         """
#         Clear the CUDA cache to free up memory.
#         """
#         print("Clearing CUDA cache...")
#         torch.cuda.empty_cache()
#         gc.collect()
#         print("CUDA cache cleared.")

#     def load_model_and_tokenizer(self, base_model_path: str, new_model_path: str) -> tuple:
#         """
#         Load the model and tokenizer.

#         Args:
#             base_model_path (str): The path to the base model.
#             new_model_path (str): The path to the new model.

#         Returns:
#             tuple: A tuple containing the loaded model and tokenizer.
#         """
#         # Load the config and base model
#         config = PeftConfig.from_pretrained(new_model_path)
#         base_model = AutoModelForCausalLM.from_pretrained(base_model_path)
#         model = PeftModel.from_pretrained(base_model, new_model_path)

#         # Load the tokenizer
#         tokenizer = AutoTokenizer.from_pretrained(new_model_path)
#         tokenizer.pad_token = tokenizer.eos_token

#         return model, tokenizer

#     def generate_response(self, model, tokenizer, prompt: str) -> str:
#         """
#         Generates a response for the given prompt using the loaded model and tokenizer.

#         Args:
#             model: The loaded language model.
#             tokenizer: The loaded tokenizer.
#             prompt (str): The input text prompt.

#         Returns:
#             str: The generated response text.
#         """
#         # Tokenize the input prompt
#         inputs = tokenizer(prompt, return_tensors="pt")

#         # Generate the response
#         outputs = model.generate(**inputs, max_length=100, do_sample=True, top_k=50, top_p=0.95)

#         # Decode the output tokens
#         response = tokenizer.decode(outputs[0], skip_special_tokens=True)

#         return response


In [1]:
from huggify_data.train_modules import *

In [2]:
from google.colab import userdata

In [3]:
# Usage
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "eagle0504/youthless-homeless-shelter-web-scrape-dataset"
new_model = "ysa-test-july-4-v3"
huggingface_token = userdata.get('HF_TOKEN')

In [4]:
trainer = LlamaTrainer(model_name, dataset_name, new_model, huggingface_token)
peft_config = trainer.configure_lora()
training_args = trainer.configure_training_arguments(num_train_epochs=1)
trainer.train_model(training_args, peft_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,2.376200


Save model...
Training and saving completed.
Starting model merging and saving...
Pushing model to the Hugging Face Hub...


adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

Model pushing completed.


In [5]:
%%time

some_model, some_tokenizer = trainer.load_model_and_tokenizer(
    base_model_path="NousResearch/Llama-2-7b-chat-hf",
    new_model_path="ysa-test-july-4-v3",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 1min 3s, sys: 18 s, total: 1min 21s
Wall time: 1min 45s


In [14]:
# Example usage
prompt = "hi, tell me a joke"
response = trainer.generate_response(some_model, some_tokenizer, prompt)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


hi, tell me a joke," the scarecrow said in the tall grass beyond the road. "I need cheering up!"
But you know what? I don't think I can do that. I have to stay serious, or I might get yelled at by the internet.
Sorry, scarecrow, I hope you find a less melancholy joke.
Labels: internet, jokes, scarecrow
The dog
